In [1]:
!nvidia-smi

Fri Aug  9 13:26:56 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P0             26W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install transformers==4.33.0 accelerate==0.22.0 einops==0.6.1 langchain==0.0.300 xformers==0.0.21 \
bitsandbytes==0.41.1 sentence_transformers==2.2.2 chromadb==0.4.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 79.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 9.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 17.6 MB/s eta 0:00:00:00:0100:

In [57]:
!wget https://www.sqlite.org/2024/sqlite-tools-linux-x64-3460000.zip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
--2024-08-09 14:14:09--  https://www.sqlite.org/2024/sqlite-tools-linux-x64-3460000.zip
Resolving www.sqlite.org (www.sqlite.org)... 45.33.6.223, 2600:3c00::f03c:91ff:fe96:b959
Connecting to www.sqlite.org (www.sqlite.org)|45.33.6.223|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11047483 (11M) [application/zip]
Saving to: 'sqlite-tools-linux-x64-3460000.zip'

sqlite-tools-linux- 100%[===================>]  10.54M  18.8MB/s    in 0.6s    

2024-08-09 14:14:09 (18.8 MB/s) - 'sqlite-tools-linux-x64-3460000.zip' saved [11047483/11047483]



In [59]:
!unzip sqlite-tools-linux-x64-3460000.zip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Archive:  sqlite-tools-linux-x64-3460000.zip
  inflating: sqlite3                 
  inflating: sqldiff                 
  inflating: sqlite3_analyzer        


In [65]:
!pip install faiss-gpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 13.9 MB/s eta 0:00:00:00:0100:01


In [3]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
#import chromadb
#from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

In [39]:
token = ""
model_name = "meta-llama/Llama-2-7b-chat-hf"

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [40]:

model_config = transformers.AutoConfig.from_pretrained(
    model_name,
    token = token
)


In [43]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
#     config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token =token
)
tokenizer = AutoTokenizer.from_pretrained(model_name,use_auth_token=token)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [44]:
time_1 = time()
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",)
time_2 = time()
print(f"Prepare pipeline: {round(time_2-time_1, 3)} sec.")

Prepare pipeline: 1.621 sec.


In [45]:
def test_model(tokenizer, pipeline, prompt_to_test):
    """
    Perform a query
    print the result
    Args:
        tokenizer: the tokenizer
        pipeline: the pipeline
        prompt_to_test: the prompt
    Returns
        None
    """
    # adapted from https://huggingface.co/blog/llama2#using-transformers
    time_1 = time()
    sequences = pipeline(
        prompt_to_test,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,)
    time_2 = time()
    print(f"Test inference: {round(time_2-time_1, 3)} sec.")
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")

In [48]:
test_model(tokenizer,
           query_pipeline,
           "What is aaruush SRM tech fest?. Give just a definition. Keep it in 100 words.")

Test inference: 5.619 sec.
Result: What is aaruush SRM tech fest?. Give just a definition. Keep it in 100 words.
Aaruush SRM Tech Fest is a platform for students to showcase their technical skills and knowledge in various fields such as artificial intelligence, machine learning, web development, and more. It provides a platform for students to learn from industry experts and network with peers and potential employers.


In [49]:
llm = HuggingFacePipeline(pipeline=query_pipeline)

llm(prompt="Please explain what is the State of the Union address. Give just a definition. Keep it in 100 words.")

'\nThe State of the Union address is an annual speech delivered by the President of the United States to a joint session of Congress, in which the President reports on the state of the union and outlines their legislative agenda for the upcoming year.'

In [50]:
loader = TextLoader("/kaggle/input/aaruush/rag.txt",
                    encoding="utf8")
documents = loader.load()

In [74]:

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

In [75]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

In [76]:
# vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")
from langchain.vectorstores import FAISS

vectordb = FAISS.from_documents(documents=all_splits, embedding=embeddings)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [77]:
retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [83]:
def test_rag(qa, query):
    print(f"Query: {query},  give me more info, generate stuff \n")
    time_1 = time()
    result = qa.run(query)
    time_2 = time()
    print(f"Inference time: {round(time_2-time_1, 3)} sec.")
    print("\nResult: ", result)

In [85]:
query = "what are the future motives of team envision?"
test_rag(qa, query)

Query: what are the future motives of team envision?,  give me more info, generate stuff 



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


> Finished chain.
Inference time: 18.473 sec.

Result:  
Team Envision is a multidisciplinary team within Aaruush, focused on providing innovative solutions to complex campus and societal issues. Their future motives include:

1. Continuing to ideate, design, fabricate, and develop products that tackle a wide range of challenges.
2. Publishing research papers and filing patents to turn their ideas into reality.
3. Collaborating with other teams within Aaruush and external partners to expand their impact.
4. Organizing workshops and events to educate and inspire others to join them in their mission to solve complex problems.
5. Fostering a culture of innovation and creativity within Aaruush and beyond.

Please let me know if you have any other questions.


In [81]:
query = "What is the mail id to team envision?"
test_rag(qa, query)

Query: What is the mail id to team envision?



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


> Finished chain.
Inference time: 9.662 sec.

Result:   I'm just an AI, I don't have access to personal information like email addresses. However, based on the context you provided, the mail id for Team Envision is most likely [envision@aaruush.org](mailto:envision@aaruush.org).


In [72]:

docs = vectordb.similarity_search(query)
print(f"Query: {query}")
print(f"Retrieved documents: {len(docs)}")
for doc in docs:
    doc_details = doc.to_json()['kwargs']
    print("Source: ", doc_details['metadata']['source'])
    print("Text: ", doc_details['page_content'], "\n")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: Who is the aaruush convener?
Retrieved documents: 4
Source:  /kaggle/input/aaruush/rag.txt
Text:  - Aaruush’s Convenor: Dr. A. Rathinam.
The Four Patrons of Aaruush are:
1. Vice-Chancellor of E&T - Dr. C. Muthamizhchelvan Sir
2. Dean - E&T - Dr. T. V. Gopal Sir
3. Convenor of Aaruush, Team Rudra, Director of DAA and Professor of EEE department - Dr. A. Rathinam Sir
4. Associate Director of Campus Life - Dr. V. Thirumurugan Sir 

Source:  /kaggle/input/aaruush/rag.txt
Text:  Organiser: Aishwaryaa Rajesh, Shivayush Mohan and Shounak Das Email: creatives@aaruush.net
5) Deco and Arts:
Deco and Arts are the committee that ensures Aaruush comes easy and pleasant to the eyes of the onlooker. Everything is covered by the Deco and Arts Committee, from artistic banners, artwork, structures, and models for decoration alongside flags, lights, and colors.
Organisers: Bhavya Kamali Vardhineedi, Sai Mahendra and S Akhilesh
Email: deco_arts@aaruush.net
6) Finance:
The Finance Committee examines